# Extract data at the intersection of a horizon and 3D volume

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

%matplotlib inline

from segysak.segy import (
    segy_loader,
    get_segy_texthead,
    segy_header_scan,
    segy_header_scrape,
)

from os import path

## Load Small 3D Volume from Volve

In [ ]:
volve_3d_path = path.join("data", "volve10r12-full-twt-sub3d.sgy")
print("3D", volve_3d_path, path.exists(volve_3d_path))

In [ ]:
get_segy_texthead(volve_3d_path)

In [ ]:
from segysak.segy import well_known_byte_locs

volve_3d = segy_loader(volve_3d_path, **well_known_byte_locs("petrel_3d"))
volve_3d.data

## Load up horizon data

In [ ]:
top_hugin_path = path.join("data", "hor_twt_hugin_fm_top.dat")
print("Top Hugin", top_hugin_path, path.exists(top_hugin_path))

In [ ]:
import pandas as pd

top_hugin_df = pd.read_csv(top_hugin_path, names=["cdp_x", "cdp_y", "twt"], sep=" ")
top_hugin_df.head()

Would be good to plot a seismic (iline,xline) section in Pyvista as well

In [ ]:
# import pyvista as pv

# point_cloud = pv.PolyData(-1*top_hugin_df.to_numpy(), cmap='viridis')
# point_cloud.plot(eye_dome_lighting=True)

Alternativey we can use the points to output a `xarray.Dataset` which comes with coordinates for plotting already gridded up for Pyvista.

In [ ]:
top_hugin_ds = volve_3d.seis.surface_from_points(
    top_hugin_df, "twt", right=("cdp_x", "cdp_y")
)
top_hugin_ds

In [ ]:
# the twt values from the points now in a form we can relate to the xarray cube.
plt.imshow(top_hugin_ds.twt)

In [ ]:
# point_cloud = pv.StructuredGrid(
#     top_hugin_ds.cdp_x.values,
#     top_hugin_ds.cdp_y.values,top_hugin_ds.twt.values,
#     cmap='viridis')
# point_cloud.plot(eye_dome_lighting=True)

## Horizon Amplitude Extraction

Extracting horizon amplitudes requires us to interpolate the cube onto the 3D horizon.

In [ ]:
top_hugin_amp = volve_3d.data.interp(
    {"iline": top_hugin_ds.iline, "xline": top_hugin_ds.xline, "twt": top_hugin_ds.twt}
)

In [ ]:
#
fig = plt.figure(figsize=(15, 5))
top_hugin_amp.plot(cmap="bwr")
cs = plt.contour(
    top_hugin_amp.xline, top_hugin_amp.iline, top_hugin_ds.twt, levels=20, colors="grey"
)
plt.clabel(cs, fontsize=14, fmt="%.1f")